# Détection de posture avec une image

## Enregistrement landmarks de l'image

In [1]:
import numpy as np
from sklearn import preprocessing
from numpy.linalg import norm
import pandas as pd
import time
import cv2
import mediapipe as mp
import csv

In [2]:
def write_landmarks(landmarks, csv_data):
    #print(f"Landmark coordinates for frame {frame_number}:")
    for idx, landmark in enumerate(landmarks):
        #print(f"{mp_pose.PoseLandmark(idx).name}: (x: {landmark.x}, y: {landmark.y}, z: {landmark.z})")
        csv_data.append([mp_pose.PoseLandmark(idx).name, landmark.x, landmark.y, landmark.z])
    print("\n")

In [ ]:
image_path = './pose1.png'
# Initialiser le module Pose de MediaPipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Lire l'image à partir du chemin spécifié
image = cv2.imread(image_path)
height, width, _ = image.shape

# Convertir l'image en RGB (MediaPipe utilise des images RGB)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Détecter les landmarks de pose dans l'image
results = pose.process(image_rgb)

list = []
write_landmarks(results.pose_landmarks.landmark, list)
list

In [ ]:
df = pd.DataFrame(list, columns=['landmarks', 'x', 'y','z'])

In [ ]:
df["x"]=((df["x"]-df["x"].min())/(df["x"].max()-df["x"].min()))
df["y"]=((df["y"]-df["y"].min())/(df["y"].max()-df["y"].min()))
df["z"]=((df["z"]-df["z"].min())/(df["z"].max()-df["z"].min()))

In [ ]:
df.to_pickle("./image.pkl")

## Comparaison et score

In [3]:
dfRef = pd.read_pickle("./image.pkl")

FileNotFoundError: [Errno 2] No such file or directory: './image.pkl'

In [ ]:
def cosine_similarity(A,B):
    return np.dot(A,B)/(norm(A)*norm(B))

#cosine_similarity(pd.concat([df["x"],df["y"]]),pd.concat([df["z"],df["y"]]))

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose #on prend ici le pose estimation model

In [ ]:
cap = cv2.VideoCapture(0)
current_seconds_time = int(time.time())
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            #'''
            if (current_seconds_time != int(time.time())):
                if (current_seconds_time % 3 == 0):
                    lis = []
                    write_landmarks(results.pose_landmarks.landmark, lis)
                    df = pd.DataFrame(lis, columns=['landmarks', 'x', 'y','z'])
                    df["x"]=((df["x"]-df["x"].min())/(df["x"].max()-df["x"].min()))
                    df["y"]=((df["y"]-df["y"].min())/(df["y"].max()-df["y"].min()))
                    print(cosine_similarity(pd.concat([dfRef["x"],dfRef["y"]]), pd.concat([df["x"],df["y"]])))
                current_seconds_time = int(time.time())
            #'''
            #landmarks = results.pose_landmarks.landmark
            #print(landmarks)
        except:
            pass
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()



0.9665382822879731


0.9734478110905067


0.9745645109257739


0.9734861136682136
